# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [63]:
import pandas as pd


bigrams_df = pd.read_csv("/kaggle/input/nlp-assignment2-data/bigrams.txt", sep='	', encoding='latin-1', header=None)

# Some words somehow convert to floats, so cast them to string first
bigrams_df[1] = bigrams_df[1].str.lower()
bigrams_df[2] = bigrams_df[2].str.lower()

bigrams_df

,0,1,2
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and
...,...,...,...
1020380,24,zviad,gamsakhurdia
1020381,25,zweimal,leben
1020382,24,zwick,and
1020383,24,zydeco,music


In [64]:
!pip install editdistance

### Reference on edit-distances: https://norvig.com/spell-correct.html

In [65]:
def edits_n(word, n=2) -> set:
    """
    Return set of words that are N edit distance away from the given word. 
    """

    e = edits1(word)
    prev = e
    for i in range(1, n):
        temp = set(e2 for e1 in prev for e2 in edits1(e1))
        e.update(temp)
        prev = temp
    e.discard(word)
    return e


def edits1(word):
    """
    All edits that are one edit away from 'word'
    """
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

    deletes = [L + R[1:] for L, R in splits if R ]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters ]
    inserts = [L + c + R for L, R in splits for c in letters]

    return set(deletes + transposes + replaces + inserts)

In [66]:
import random


class SentenceGenerator:
    """
    This class generates error sentences.
    
    Why you cannot make them yourself? 
    SpellingChecker assumes the error word is within 2 edit distances from the original one,
    so it better to have a separate class for satisfying our assumption.
    """
    
    def __init__(self, train_data: dict):
        self.train_data = train_data
    
    
    def generate_error_word(self, word, edit_distance=2):
        """
        Function for generating test dataset.
        Generate error word within edit distance of the original word
        """
        
        possible_errors = edits_n(word, edit_distance)
        
        non_word_errors = []
        real_word_errors = []
        for error in possible_errors:
            if error in self.train_data:
                real_word_errors.append(error)
            else:
                non_word_errors.append(error)
        
        # With 50% probability choose either 
        # non-word or real-word error
        c = random.randint(0,1)
        if c or len(real_word_errors)==0:
            return random.choice(non_word_errors)
        
        return random.choice(real_word_errors)
    
    def generate_error_sentences(self, sentences: [str], edit_distance = 2, errors_per_sentence = 1) -> [[str]]:
        """
        Input is the correct sentence without errors.
        
        Assumes that the input is the set of sentences (strings).
        Sentences should not include the punctuation.
        
        Returns sentences with different errors (which might be non-sense too).
        """
        
        output = []
        for sentence in sentences:
            tokenized_data = sentence.lower().split(" ")
                        
            # Randomly shuffle words in the sentence and iterate over them.
            indices = random.choices(range(len(tokenized_data)), k = errors_per_sentence)
            
            for i in indices:
                tokenized_data[i] = self.generate_error_word(tokenized_data[i], edit_distance)
            
            output.append(tokenized_data)
                        
        return output

In [67]:
from math import log10, inf


class Ngram:
    def __init__(self, grams=2):
        self.n = grams
        self.train_data = {}
        self.max_count = 0
    
    def train(self, train_df: pd.DataFrame):
        for index, row in train_df.iterrows():
            count = row[0]
            word1 = row[1]
            word2 = row[2]

            if word1 not in self.train_data:
                self.train_data[word1] = []

            self.train_data[word1].append((word2, count))
            self.max_count += count
    
    def get_probability(self, context: str, word: str) -> float: 
        """
        Returns probability of `word` in the given context.
        Context is the string consisting of single word.
        """            
        
        count_pair = self.is_word_in_train_data_context(context, word)
        
        if count_pair:
            return log10(count_pair[1] / self.max_count)
        
        # If there is no such context or word in the dictionary, then we assign minimum probability
        return -999
    
    def is_word_in_train_data_context(self, context: str, word: str):
        """
        Returns pair of (word, count) if the given context (key in dict)
        contains the given word.
        """
        
        # If there is no such context
        if context not in self.train_data:
            return None
        
        for count_pair in self.train_data[context]:
            if count_pair[0] != word:
                continue
                
            return count_pair
        
        return None

In [68]:
import editdistance


class SpellingChecker(Ngram):
    """
    Class that implements 2-gram spell checker, given the training data consisting of 3 columns:
    (count, word1, word2).
    """
    
    def __init__(self, grams=2):
        """
        Initialises train data, which is a dictionary, where each key is a word from column 1 (word1),
        and value is a list of tuples (word2, count).
        """
        
        super().__init__(grams)

    def get_corrected_word(self, context, word, next_word="") -> str:
        """
        Find the most-probable correction for a word, combinining:
        (context + word) and (word + next word) as max probability.
        """
        
        corrected_word = word
        max_prob = self.get_probability(context, word) + self.get_probability(word, next_word)
                
        # Calculate probability for every word in dictionary
        # if it is 2 edit-distances away.
        for candidate_word in self.train_data:
            context_prob = self.get_probability(context, candidate_word)
            word_prob = self.get_probability(candidate_word, next_word)

            prob_sum = context_prob + word_prob

            if prob_sum > max_prob:
                max_prob = prob_sum
                corrected_word = candidate_word
                                    
        return corrected_word
    
    def find_and_correct_errors(self, data_with_errors: [[str]], max_to_correct = 1) -> [[str]]:
        """
        Detect and correct all errors in a list of sentences.
        """
        
        corrected_data = data_with_errors
        
        # Iterate through sentences
        for j in range(len(corrected_data)):
            context = ''
            words_corrected = 0
            
            # We will use these 2 values for
            # identifying the least probable word, and correct it, in case all
            # words are in the dictionary.
            min_probability = 999999
            min_word_index = None
            
            # Iterate through words
            for i in range(len(corrected_data[j]) - 1):                
                if words_corrected == max_to_correct:
                    break
                
                next_word_after_error = corrected_data[j][i + 1]
                
                context_prob = self.get_probability(context, corrected_data[j][i])
                
                # Assumption: if the word is not in the train_data,
                # it is most likely an error word.
                if(corrected_data[j][i] not in self.train_data):
                                        
                    word_before_correction = corrected_data[j][i]
                    corrected_data[j][i] = self.get_corrected_word(context, corrected_data[j][i], next_word_after_error)
                                        
                    if word_before_correction != corrected_data[j][i]:
                        words_corrected += 1
                    
                context = corrected_data[j][i]
                
                next_prob = self.get_probability(context, next_word_after_error)
                
                prob_sum = context_prob + next_prob
                if prob_sum < min_probability:
                    min_probability = prob_sum
                    min_word_index = i
                
            # If all words are in the train data, then
            # find and fix the least probable word
            if min_word_index and words_corrected == 0:
                
                # Some checks for accessing the context and next word
                context = ''
                if min_word_index > 1:
                    context = corrected_data[j][min_word_index - 1]
                
                next_word_after_error = ''
                if min_word_index + 1 < len(corrected_data[j]):
                    next_word_after_error = corrected_data[j][min_word_index + 1]
                
                before = corrected_data[j][min_word_index]
                corrected_data[j][min_word_index] = self.get_corrected_word(context, corrected_data[j][min_word_index], next_word_after_error)
                        
        return corrected_data

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

### At first, I have used bigrams dataset provided in the assignment description, as larger grams tend to lead to slower train time and requires much larger sets of data to be accurate.

### Secondly, I distinguish the words that are present in the train data, and not present, which partially mitigates the problem with large set of data. However, despite that, my solution still depends on large corpus.

### Lastly, I am mainly use bigram model to correct the spelling as it is the fastest method, but maybe not the most accurate one. Still, it checks the context of the previous and next words from error word, which it already better than the Norvig's solution.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [69]:
bigram_model = SpellingChecker()
bigram_model.train(bigrams_df)

### Probibilities test

In [70]:
print(bigram_model.get_probability("a", ""))
print(bigram_model.get_probability("a", "a"))
print(bigram_model.get_probability("a", "aaa"))

assert bigram_model.get_probability("a", "a") > bigram_model.get_probability("a", "aaa")

-999
-6.018183160884371
-6.966154160880362


### Small test for 'I' correction

In [71]:
error_data = [['eig', 'am', 'a', 'responsible', 'person']]
bigram_model.find_and_correct_errors(error_data)

[['i', 'am', 'a', 'responsible', 'person']]

In [72]:
sentence_generator = SentenceGenerator(bigram_model.train_data)
input_sentences = ["I am a responsible person", "Hi my name is John", "Hey buddy what are you doing here", "NLP course is a good one"]
error_sentences = sentence_generator.generate_error_sentences(input_sentences)
error_sentences

[['hr', 'am', 'a', 'responsible', 'person'],
 ['hi', 'my', 'name', 'qiss', 'john'],
 ['hey', 'buddy', 'what', 'are', 'ion', 'doing', 'here'],
 ['nlp', 'copse', 'is', 'a', 'good', 'one']]

In [73]:
corrected_data = bigram_model.find_and_correct_errors(error_sentences)
corrected_data

[['hr', 'am', 'a', 'responsible', 'person'],
 ['hi', 'my', 'name', 'of', 'john'],
 ['hey', 'buddy', 'what', 'are', 'you', 'doing', 'here'],
 ['a', 'copse', 'is', 'a', 'good', 'one']]

### In the Norvig's solution there is a abscence of the context checking, which my solution successfully provides with the use of bigrams.

### Next, Nordwig provides only 2 edit distances spell checker, while my solution iterates over all possible edit distances and checks if these words are in the train data, which makes my model more accurately predict the correction.